In [ ]:
import pandas as pd
import pandas_datareader as reader
import datetime as dt
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_excel("/content/1564773405.xlsx")
df = df.rename(columns={"Unnamed: 0": "sign"})

In [ ]:
df = df[df["sign"]=='-']
df

,sign,Company,Code,Date
0,-,Concho Resources,CXO.N^A21 (expired),2021-01-21
3,-,Tiffany,TIF.N^A21 (expired),2021-01-07
4,-,Aimco,AIV.N,2020-12-21
5,-,Apartment Income,AIRC.N,2020-12-21
8,-,Noble Energy,NBL.OQ^J20 (expired),2020-10-12
...,...,...,...,...
1443,-,Lockheed,LK.N^C95 (expired),1995-03-16
1444,-,Martin Maretta,ML.N^C95 (expired),1995-03-16
1446,-,Natl Education,NEC.N^F97 (expired),1995-02-24
1448,-,Pet,PT.N^E95 (expired),1995-02-09


In [ ]:
df['start_date'] = df['Date'] - pd.Timedelta(days=21)
df['start_date'] = df['start_date'] - pd.DateOffset(years=3)
df['end_date'] = df['Date'] + pd.Timedelta(days=21)
df = df[df['end_date'] <= '2020-12-31']

In [ ]:
df['ticker'] = df['Code'].apply(lambda x: x.split('.', 1)[0])
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sign,Company,Code,Date,start_date,end_date,ticker
8,-,Noble Energy,NBL.OQ^J20 (expired),2020-10-12,2017-09-21,2020-11-02,NBL
10,-,E*TRADE Financl,ETFC.OQ^J20 (expired),2020-10-07,2017-09-16,2020-10-28,ETFC
13,-,Coty,COTY.N,2020-09-21,2017-08-31,2020-10-12,COTY
15,-,H&R Block,HRB.N,2020-09-21,2017-08-31,2020-10-12,HRB
16,-,Kohls,KSS.N,2020-09-21,2017-08-31,2020-10-12,KSS
...,...,...,...,...,...,...,...
1443,-,Lockheed,LK.N^C95 (expired),1995-03-16,1992-02-23,1995-04-06,LK
1444,-,Martin Maretta,ML.N^C95 (expired),1995-03-16,1992-02-23,1995-04-06,ML
1446,-,Natl Education,NEC.N^F97 (expired),1995-02-24,1992-02-03,1995-03-17,NEC
1448,-,Pet,PT.N^E95 (expired),1995-02-09,1992-01-19,1995-03-02,PT


In [ ]:
# prices = reader.get_data_yahoo("Pool", start=df.iloc[0]['start_date'], end=df.iloc[0]['end_date'])['Close']
# if prices.index[0] >= df.iloc[0]['start_date']+pd.Timedelta(days=10):
#     raise ValueError('Not old enough')
# daily_return = np.log((prices/prices.shift(1)).dropna())
# fama = reader.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', df.iloc[0]['start_date'], df.iloc[0]['end_date'])[0][1:]
# daily_return = pd.merge(daily_return, fama, on='Date')
# daily_return['y'] = daily_return['Close'] - daily_return['RF']
# daily_return

In [ ]:
# train = daily_return.iloc[:-43]
# test = daily_return.iloc[-43:]
# y = train['y']
# X = train[["Mkt-RF", "SMB", "HML"]]
# model = sm.OLS(y, X)
# results = model.fit()

In [ ]:
# test_X = test[["Mkt-RF", "SMB", "HML"]]
# print(list(results.predict(test_X)))

In [ ]:
success = []
failures = []

for i in range(len(df.index)):
    ticker = df.iloc[i]['ticker']
    start = df.iloc[i]['start_date']
    end = df.iloc[i]['end_date']
    date = df.iloc[i]['Date']
    
    try:
        prices = reader.get_data_yahoo(ticker, start=start, end=end)['Close']
        if prices.index[0] >= start + pd.Timedelta(days=10):
            raise ValueError('Not old enough')
        daily_return = np.log((prices/prices.shift(1)).dropna())
        fama = reader.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start, end)[0][1:]
        daily_return = pd.merge(daily_return, fama, on='Date')
        daily_return['y'] = daily_return['Close'] - daily_return['RF']
        
        train = daily_return.iloc[:-43]
        test = daily_return.iloc[-43:]
        y = train['y']
        X = train[["Mkt-RF", "SMB", "HML"]]
        model = sm.OLS(y, X)
        results = model.fit()

        test_X = test[["Mkt-RF", "SMB", "HML"]]
        test_y = test['y']
        preds = results.predict(test_X)
        abnormal = test_y - preds

        success.append((ticker, list(abnormal),
                        results.params[0], results.params[1], results.params[2]))
        print(i, ticker, "success")

        
    except:
        failures.append(ticker)
        print(i, ticker, "failure")

In [ ]:
success_df = pd.DataFrame(success,  columns=['Ticker', 'abnormal', 'Mkt-RF', 'SMB', 'HML'])
success_df.to_excel("output_minus.xlsx")

In [ ]:
failures

In [ ]:
success_df

,Ticker,abnormal,Mkt-RF,SMB,HML
0,COTY,"[-0.0008488589717072755, -0.000508466666982912...",0.010535,0.010419,0.012359
1,HRB,"[-0.024109353151822943, -0.008347956665231126,...",0.009475,0.004538,0.007563
2,KSS,"[0.018830786879012525, -0.008693387254967561, ...",0.011116,0.009395,0.016724
3,ADS,"[-0.007793896593688163, -0.027148682644557387,...",0.013256,0.012215,0.011481
4,HOG,"[0.03502819237256789, 0.014097196107565255, 0....",0.012701,0.003317,0.012233
...,...,...,...,...,...
207,ROK,"[-0.020614629121486417, 0.006835264365321255, ...",0.006314,-0.002379,0.000592
208,FNB,"[-0.022366211698898714, -0.03133061775094711, ...",0.005489,0.008290,0.007944
209,BSET,"[-0.0030054138394209158, -0.04190794630992603,...",0.001206,-0.004249,-0.003392
210,SKY,"[-0.01392271775342536, -0.03628537843904696, -...",0.007989,0.001030,0.000713
